In [1]:
import os
from env_config import set_env_vars

# --- Load environment variables and secrets ---
set_env_vars()

2025-06-09 21:15:35,109 - INFO - Loaded local .env file from /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-09 21:15:35,109 - INFO - Running in context: CLI
2025-06-09 21:15:35,109 - INFO - Force authentication: False


In [2]:
from gcp_utils import get_gcp_credentials, init_sheets_client, init_drive_client
from qdrant_utils import init_qdrant_client

creds = get_gcp_credentials()
sheets_client = init_sheets_client(creds)
drive_client = init_drive_client(creds)
qdrant_client = init_qdrant_client("cloud")

2025-06-09 21:15:37,025 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-06-09 21:15:37,027 - INFO - Initializing Qdrant client in 'cloud' mode...
2025-06-09 21:15:37,488 - INFO - HTTP Request: GET https://0c82e035-1105-40f2-a0bd-ecc44a016f15.us-east4-0.gcp.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
2025-06-09 21:15:37,566 - INFO - HTTP Request: GET https://0c82e035-1105-40f2-a0bd-ecc44a016f15.us-east4-0.gcp.cloud.qdrant.io:6333/collections "HTTP/1.1 200 OK"
2025-06-09 21:15:37,568 - INFO - Available Qdrant collections: ['ASK_vectorstore', 'ask_pdf_docs', 'ASK_vectorstore-backup21APR2025', 'ask_pdf_pages']
2025-06-09 21:15:37,568 - INFO - ✅ Confirmed Qdrant collection 'ASK_vectorstore' exists


### Run your functions to test in the cells below

Delete tagged


In [ ]:
from delete_tagged import delete_tagged


deleted_rows_df = delete_tagged(drive_client, sheets_client, qdrant_client)

Validate row

In [3]:
from gcp_utils import fetch_sheet_as_df
from library_utils import validate_all_rows_format
from IPython.display import display, HTML


library_df = fetch_sheet_as_df(sheets_client, os.environ["LIBRARY_UNIFIED"])
valid_df, invalid_df, log_df = validate_all_rows_format(library_df)

print(f"\nInvalid rows:")
if not invalid_df.empty:
    display(HTML(invalid_df.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

2025-06-09 21:15:40,080 - INFO - ✅ Found 221 valid rows in LIBRARY_UNIFIED.
2025-06-09 21:15:40,080 - INFO - ⚠️ Found 2 invalid rows in LIBRARY_UNIFIED.



Invalid rows:


,issues,title,pdf_id,publication_number,organization,scope,unit,issue_date,upsert_date,expiration_date,aux_specific,public_release,pdf_file_name,embedding,google_id,link,status,status_timestamp
166,"missing_required_fields: ['google_id', 'link']","COAST GUARD EXTERNAL AFFAIRS MANUAL, COMDTINST...",d354afd1-95ea-5489-9df7-3abdd322d73c,COMDTINST M5700.13A,CG-092,National,,2021-05-12T00:00:00Z,,2031-05-13T00:00:00Z,FALSE,TRUE,CG Public Affairs CIM_5700_13A_2021-05-01.pdf,text-embedding-ada-002,,,orphan_row,
222,"missing_required_fields: ['title', 'scope', 'i...",,6b32e367-c17c-52bb-b913-e6452d5fe307,,,,,,,,,,Waxtree Ligstrum Privet floral accord solution...,,1acYv9NONs4yx8eND59t3y5vnS_jEVM7G,https://drive.google.com/file/d/1acYv9NONs4yx8...,,2025-05-23T18:39:11Z


cleanup orphans

In [ ]:
from cleanup_orphans import find_orphans
from IPython.display import display, HTML


orphan_rows, orphan_files, orphan_qdrant_records, log_df = find_orphans(
    drive_client, sheets_client, qdrant_client)

print(f"\nOrphan rows:")
if not orphan_rows.empty:
    display(HTML(orphan_rows.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

print(f"\nOrphan files:")
if not orphan_files.empty:
    display(HTML(orphan_files.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

print(f"\nOrphan qdrant records:")
if not orphan_qdrant_records.empty:
    display(HTML(orphan_qdrant_records.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))